In [2]:
import pandas as pd

df_da = pd.read_csv("fbref/Big5DA1920.csv",header=[0,1])
df_sca = pd.read_csv("fbref/Big5SCA1920.csv",header=[0,1])

def rename_columns(df):
    new_cols = []
    for col in df.columns:
        if "Unnamed:" in col[0]:
            new_cols.append(col[1])
        else:
            new_cols.append(col[0] + " " + col[1])
    df.columns = new_cols
    df["Player"] = df["Player"].apply(lambda x: x.split("\\")[0])
    # df["Nation"] = df["Nation"].apply(lambda x: x.split(" ")[0])
    df = df.drop(columns=['Matches'])
    return df
    
df_da = rename_columns(df_da)
df_sca = rename_columns(df_sca)

In [3]:
# print(df_da.columns)
# print(df_sca.columns)
drop_cols = ['Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s']
df_all = pd.concat([df_da, df_sca.drop(columns=drop_cols)], axis=1, ignore_index=True)
df_all.columns = df_da.columns.tolist() + df_sca.drop(columns=drop_cols).columns.tolist()
print(df_all.columns)

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s',
       'Tackles Tkl', 'Tackles TklW', 'Tackles Def 3rd', 'Tackles Mid 3rd',
       'Tackles Att 3rd', 'Vs Dribbles Tkl', 'Vs Dribbles Att',
       'Vs Dribbles Tkl%', 'Vs Dribbles Past', 'Pressures Press',
       'Pressures Succ', 'Pressures %', 'Pressures Def 3rd',
       'Pressures Mid 3rd', 'Pressures Att 3rd', 'Blocks Blocks', 'Blocks Sh',
       'Blocks ShSv', 'Blocks Pass', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Rk',
       'SCA SCA', 'SCA SCA90', 'SCA Types PassLive', 'SCA Types PassDead',
       'SCA Types Drib', 'SCA Types Sh', 'SCA Types Fld', 'GCA GCA',
       'GCA GCA90', 'GCA Types PassLive', 'GCA Types PassDead',
       'GCA Types Drib', 'GCA Types Sh', 'GCA Types Fld', 'GCA Types OG'],
      dtype='object')


In [4]:
df_mffw = df_all[(df_all["90s"]>=15) 
       & (df_all["Pos"].str.contains("MF") | df_all["Pos"].str.contains("FW"))].copy()

In [5]:
df_mffw["Pressures Att 3rd 90"] = df_mffw["Pressures Att 3rd"] / df_mffw["90s"]
df_mffw["Pressures Mid 3rd 90"] = df_mffw["Pressures Mid 3rd"] / df_mffw["90s"]
df_mffw["SCA Types PassLive 90"] = df_mffw["SCA Types PassLive"] / df_mffw["90s"]
df_mffw["Pressures Mid Att 90"] = \
    (df_mffw["Pressures Att 3rd"] + df_mffw["Pressures Mid 3rd"])/ df_mffw["90s"]
df_mffw["Tackles Mid Att 90"] = \
    (df_mffw["Tackles Att 3rd"] + df_mffw["Tackles Mid 3rd"])/ df_mffw["90s"]
df_mffw["Blocks Pass 90"] = df_mffw["Blocks Pass"] / df_mffw["90s"]
df_mffw["Int 90"] = df_mffw["Int"] / df_mffw["90s"]
df_mffw["DA 90"] = df_mffw["Pressures Mid Att 90"] +\
                   df_mffw["Tackles Mid Att 90"] +\
                   df_mffw["Int 90"] + df_mffw["Blocks Pass 90"]

In [6]:
cols = ["Player","Squad","Age","Pos","90s",
        "DA 90","SCA Types PassLive 90"]
plot_df = df_mffw[((df_mffw["Pos"].str.contains("MF")) ) & (
    df_mffw["SCA Types PassLive 90"]>=2
    )]
plot_df[cols].sort_values(
    by=["DA 90","SCA Types PassLive 90"],
    ascending=False).head(50)

,Player,Squad,Age,Pos,90s,DA 90,SCA Types PassLive 90
2148,Valentin Rongier,Marseille,24.0,MF,19.9,30.251256,2.010050
316,Jean-Paul Boëtius,Mainz 05,25.0,MF,25.2,28.333333,2.341270
2230,Morgan Sanson,Marseille,24.0,MF,24.1,27.842324,2.157676
2082,Marco Reus,Dortmund,30.0,MFFW,16.6,27.409639,2.469880
1929,Mathias Pereira Lage,Angers,22.0,MF,17.1,27.192982,2.163743
305,Ludovic Blas,Nantes,21.0,FWMF,20.5,26.731707,2.195122
2467,Adrien Thomasson,Strasbourg,25.0,MF,23.0,26.043478,2.565217
2390,Lars Stindl,M'gladbach,30.0,MFFW,15.9,25.408805,3.270440
302,Leonardo Bittencourt,Werder Bremen,25.0,FWMF,21.0,24.619048,2.285714
1935,Ayoze Pérez,Leicester City,26.0,MFFW,22.3,24.573991,2.376682


In [8]:
# df_mffw[(df_mffw["Player"]
#     .str.contains("Partey")) | (
#     df_mffw["Squad"]
#     .str.contains("Arsenal")
#     )][cols]

In [9]:
# df_mffw[(
#     df_mffw["Squad"]
#     .str.contains("Manchester City")
#     )][cols]

In [80]:
df_mffw[cols].mean()

Age                      25.954058
90s                      23.318836
Pressures Mid 3rd 90      9.082939
SCA Types PassLive 90     1.765845
dtype: float64

In [10]:
import matplotlib.pyplot as plt
import numpy as np

print(len(plot_df))
x = plot_df["Pressures Mid Att 90"].values
y = plot_df["SCA Types PassLive 90"].values
# print(npxGpSh.max())
# print(npxGcon.max())

fig=plt.figure(figsize=(8,6))
fig.set_facecolor("white")
ax=fig.add_axes([0,0,1,1])
# ax.set_yticks(np.arange(0,20,2))
ax.scatter(x, y, color='grey')
ax.set_xlabel('Pressures')
ax.set_ylabel('SCA')
ax.set_title('ABC')
plt.grid(True)
for x1, y1, c1,t1 in zip(x,y,plot_df["Squad"].values,plot_df["Player"].values):
    if c1=="Arsenal":
        ax.scatter(x1, y1, color='r')
    if  (y1>3) | ((y1>3) & (x1>15)) \
            | (x1>20):
        ax.annotate(t1, (x1, y1))
    if (c1=="Arsenal"):
        ax.annotate(t1, (x1, y1))        

# for i, txt in enumerate(plot_df["Player"].values):
# z = np.polyfit(x, y, 1)
# p = np.poly1d(z)
# plt.plot(x,p(X),"r-")
plt.show()

169


<Figure size 800x600 with 1 Axes>